## Creating a WebMap from the MapViewer Widget


### Set your own properties

In [ ]:
gemeente = "Bronckhorst"

#### Connect to your ArcGIS Portal

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

#### Let's get some layers to add to the map

In [ ]:
gemeentehuizenItem = gis.content.get("9b231e5a9d984d398f552eceaddaca21")
gemeentehuizenLayer = gemeentehuizenItem.layers[0]
print(f"Found layer: {gemeentehuizenLayer.properties.name}")

In [ ]:
gemeenteGrenzenItem = gis.content.get("96e45f8f0f304bfd84a8ab58edfacbd7")
gemeenteGrenzenLayer = gemeenteGrenzenItem.layers[0]
print(f"Found layer: {gemeenteGrenzenLayer.properties.name}")

#### Create a new Map

In [ ]:
newMap = gis.map(f"{gemeente}, Netherlands")

#### Get the results from the Gemeenteraadverkiezingen 2022

In [ ]:
detailsVerkiezingen = gis._con.get("https://www.verkiezingsuitslagen.nl/verkiezingen/detailJson/GR20220316")
presentGemeentes = {regio["Value"]: regio["Id"] for regio in detailsVerkiezingen["Regios"]["Regios"][0]["Options"]}

In [ ]:
if gemeente in presentGemeentes:
    gemeenteId = presentGemeentes[gemeente]
    uitslagJson = gis._con.get(f"https://www.verkiezingsuitslagen.nl/verkiezingen/detailJson/GR20220316/{gemeenteId}")
    winner = uitslagJson["PieChart"][0]["Name"]
    colorHex = uitslagJson["PieChart"][0]["Color"]
    if not colorHex:
        color = [247,163,92]
    else:
        color = [int(colorHex[1:][i:i+2], 16) for i in (0, 2, 4)]
else:
    winner = "Nog niet bekend"
    color = [255, 255, 255]
print(f"The winner in Gemeente {gemeente} is {winner}")

#### Add a graphic with the <i>Gemeentegrens</i> to the map, setting the color to the winner

In [ ]:
newMap.clear_graphics()

selectedGemeente = gemeenteGrenzenLayer.query(f"statnaam = '{gemeente}'", out_fields=["statnaam"])
selectedGemeente.features[0].attributes["winner"] = f"De grootste partij is: {winner}"
newMap.draw(selectedGemeente, attributes={"winner": winner}, popup={"title": "statnaam", "content": "winner"}, symbol={
      "type": "esriSFS",
      "style": "esriSFSSolid",
      "color": color,
        "outline": {
         "type": "esriSLS",
         "style": "esriSLSSolid",
         "color": [110,110,110,100],
         "width": 1
        }
    })

#### Add the layer with the <i>Gemeentehuizen</i> to the map, using a Definition Expression

In [ ]:
newMap.add_layer(gemeentehuizenLayer, {"definition_expression": f"gemeente = '{gemeente}'"})

#### View the result

In [ ]:
display(newMap)

#### Save the map as a new WebMap Item in your Content

In [ ]:
webmapProperties = {
    "title": f"Gemeenteraadsverziekingen 2022: {gemeente}",
    "snippet": "WebMap created from MapViewer Widget in a Notebook",
    "tags": ["automation", "python", "MapViewer", "Notebook", gemeente]
}
webmapItem = newMap.save(webmapProperties)
display(webmapItem)